# COMP 432 - BIXI

In [143]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
from tqdm import tqdm

# Removing scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## Load BIXI data

In [2]:
zf = ZipFile('./data/bixi/biximontrealrentals2019-33ea73.zip')
bixi_2019_04 = pd.read_csv(zf.open('OD_2019-04.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_05 = pd.read_csv(zf.open('OD_2019-05.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_06 = pd.read_csv(zf.open('OD_2019-06.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_07 = pd.read_csv(zf.open('OD_2019-07.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_08 = pd.read_csv(zf.open('OD_2019-08.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_09 = pd.read_csv(zf.open('OD_2019-09.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_10 = pd.read_csv(zf.open('OD_2019-10.csv'), parse_dates=['start_date', 'end_date'])
bixi_2019_stations = pd.read_csv(zf.open('Stations_2019.csv'), dtype={'name': 'string'})

/Users/simon/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
bixi_2019_04.head()

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
0,2019-04-14 07:55:22,6001,2019-04-14 08:07:16,6132,713,1
1,2019-04-14 07:59:31,6411,2019-04-14 08:09:18,6411,587,1
2,2019-04-14 07:59:55,6097,2019-04-14 08:12:11,6036,736,1
3,2019-04-14 07:59:57,6310,2019-04-14 08:27:58,6345,1680,1
4,2019-04-14 08:00:37,7029,2019-04-14 08:14:12,6250,814,0


In [4]:
bixi_2019_stations.head()

,Code,name,latitude,longitude
0,10002,Métro Charlevoix (Centre / Charlevoix),45.478,-73.570
1,4000,Jeanne-d'Arc / Ontario,45.550,-73.542
2,4001,Graham / Brookfield,45.520,-73.630
3,4002,Graham / Wicksteed,45.517,-73.640
4,5002,St-Charles / Montarville,45.534,-73.515


## Load weather data

In [5]:
# Note: Use Date/Time column instead of Time column
weather_2019_04 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_04-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})
weather_2019_05 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_05-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})
weather_2019_06 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_06-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})
weather_2019_07 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_07-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})
weather_2019_08 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_08-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})
weather_2019_09 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_09-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})
weather_2019_10 = pd.read_csv('./data/weather/2019/en_climate_hourly_QC_7025251_10-2019_P1H.csv', parse_dates=['Date/Time'], dtype={'Station Name': 'string', 'Weather': 'string'})

In [6]:
weather_2019_04.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 00:00:00,2019,4,1,00:00,-1.500,...,nan,24.100,nan,100.750,nan,nan,nan,-7.000,nan,Snow
1,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 01:00:00,2019,4,1,01:00,-1.700,...,nan,24.100,nan,100.790,nan,nan,nan,-9.000,nan,Cloudy
2,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 02:00:00,2019,4,1,02:00,-2.900,...,nan,24.100,nan,100.860,nan,nan,nan,-9.000,nan,<NA>
3,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 03:00:00,2019,4,1,03:00,-3.900,...,nan,24.100,nan,100.920,nan,nan,nan,-10.000,nan,<NA>
4,-73.740,45.470,MONTREAL INTL A,7025251,2019-04-01 04:00:00,2019,4,1,04:00,-4.800,...,nan,24.100,nan,101.030,nan,nan,nan,-11.000,nan,Clear


## Merging all data together

### BIXI data

Merging all 2019 bixi trips together

In [7]:
bixi_2019_all = pd.concat([bixi_2019_04, bixi_2019_05, bixi_2019_06, bixi_2019_07, bixi_2019_08, bixi_2019_09, bixi_2019_10])
bixi_2019_all

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member
0,2019-04-14 07:55:22,6001,2019-04-14 08:07:16,6132,713,1
1,2019-04-14 07:59:31,6411,2019-04-14 08:09:18,6411,587,1
2,2019-04-14 07:59:55,6097,2019-04-14 08:12:11,6036,736,1
3,2019-04-14 07:59:57,6310,2019-04-14 08:27:58,6345,1680,1
4,2019-04-14 08:00:37,7029,2019-04-14 08:14:12,6250,814,0
...,...,...,...,...,...,...
617190,2019-10-31 23:52:45,6901,2019-11-01 00:05:15,6063,749,1
617191,2019-10-31 23:53:59,6218,2019-11-01 00:23:19,6269,1760,1
617192,2019-10-31 23:55:26,7149,2019-11-01 00:11:33,6338,966,1
617193,2019-10-31 23:56:28,6199,2019-11-01 00:09:13,6003,764,1


Append station data to all Bixi Trips

***May take a while to run**

In [16]:
bixi_2019_all_with_stations = bixi_2019_all.merge(bixi_2019_stations, left_index=True, left_on='start_station_code', right_on='Code').drop('Code', 1).rename({'name': 'start_name', 'latitude': 'start_latitude', 'longitude': 'start_longitude'}, axis=1).merge(bixi_2019_stations, left_index=True, left_on='end_station_code', right_on='Code').drop('Code', 1).rename({'name': 'end_name', 'latitude': 'end_latitude', 'longitude': 'end_longitude'}, axis=1)
bixi_2019_all_with_stations.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,end_latitude,end_longitude
328,2019-10-03 14:56:26,6175,2019-10-03 15:15:00,6372,1113,1,St-Christophe / Cherrier,45.520,-73.568,1ère avenue / Masson,45.545,-73.576
93,2019-09-03 12:01:46,6072,2019-09-03 12:10:17,6096,511,1,Metcalfe / de Maisonneuve,45.502,-73.574,Lucien L'Allier / St-Jacques,45.494,-73.569
132,2019-05-08 10:26:20,6907,2019-05-08 10:31:43,6136,322,1,Boyer / Rosemont,45.534,-73.595,Métro Laurier (Rivard / Laurier),45.528,-73.589
93,2019-10-10 17:21:36,6090,2019-10-10 17:23:00,6096,83,1,Métro Lucien-L'Allier (Lucien l'Allier / Argyle),45.495,-73.571,Lucien L'Allier / St-Jacques,45.494,-73.569
451,2019-08-06 20:22:01,6157,2019-08-06 20:37:39,6913,937,1,de Brébeuf / du Mont-Royal,45.529,-73.578,Drolet / Beaubien,45.533,-73.606


### Weather

Merging weather data

In [9]:
weather_2019_all = pd.concat([weather_2019_04, weather_2019_05, weather_2019_06, weather_2019_07, weather_2019_08, weather_2019_09, weather_2019_10])
weather_2019_all.sample(10)

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
323,-73.740,45.470,MONTREAL INTL A,7025251,2019-06-14 11:00:00,2019,6,14,11:00,14.600,...,nan,48.300,nan,100.000,nan,nan,nan,nan,nan,Rain Showers
426,-73.740,45.470,MONTREAL INTL A,7025251,2019-07-18 18:00:00,2019,7,18,18:00,26.900,...,nan,24.100,nan,100.700,nan,29.000,nan,nan,nan,<NA>
606,-73.740,45.470,MONTREAL INTL A,7025251,2019-09-26 06:00:00,2019,9,26,06:00,19.100,...,nan,48.300,nan,99.920,nan,nan,nan,nan,nan,<NA>
130,-73.740,45.470,MONTREAL INTL A,7025251,2019-06-06 10:00:00,2019,6,6,10:00,18.900,...,nan,48.300,nan,100.550,nan,nan,nan,nan,nan,Clear
381,-73.740,45.470,MONTREAL INTL A,7025251,2019-09-16 21:00:00,2019,9,16,21:00,15.100,...,nan,24.100,nan,101.440,nan,nan,nan,nan,nan,<NA>
156,-73.740,45.470,MONTREAL INTL A,7025251,2019-08-07 12:00:00,2019,8,7,12:00,23.300,...,nan,24.100,nan,100.420,nan,30.000,nan,nan,nan,<NA>
154,-73.740,45.470,MONTREAL INTL A,7025251,2019-06-07 10:00:00,2019,6,7,10:00,20.400,...,nan,48.300,nan,100.890,nan,nan,nan,nan,nan,Clear
274,-73.740,45.470,MONTREAL INTL A,7025251,2019-09-12 10:00:00,2019,9,12,10:00,16.600,...,nan,24.100,nan,102.050,nan,nan,nan,nan,nan,Mostly Cloudy
415,-73.740,45.470,MONTREAL INTL A,7025251,2019-05-18 07:00:00,2019,5,18,07:00,9.900,...,nan,48.300,nan,101.070,nan,nan,nan,nan,nan,Mostly Cloudy
451,-73.740,45.470,MONTREAL INTL A,7025251,2019-06-19 19:00:00,2019,6,19,19:00,25.000,...,nan,24.100,nan,100.180,nan,29.000,nan,nan,nan,Mostly Cloudy


### BIXI + Weather

Merging weather data (Tempeture and weather) with BIXI data

In [10]:
bixi_2019_all_with_stations_and_weather = bixi_2019_all_with_stations.merge(weather_2019_all[['Date/Time', 'Weather', 'Temp (°C)']], left_on=bixi_2019_all_with_stations['start_date'].dt.floor('H'), right_on='Date/Time').drop('Date/Time', 1).rename({'Weather': 'start_weather', 'Temp (°C)': 'start_temp'}, axis=1)
bixi_2019_all_with_stations_and_weather.sample(10)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,end_latitude,end_longitude,start_weather,start_temp
2204718,2019-08-04 16:07:03,6023,2019-08-04 16:36:10,6921,1746,0,de la Commune / Berri,45.511,-73.550,Augustin-Cantin / Shearer,45.484,-73.563,Clear,23.900
2494602,2019-08-25 14:33:42,6205,2019-08-25 14:49:26,6023,943,1,Milton / du Parc,45.509,-73.574,de la Commune / Berri,45.511,-73.550,<NA>,24.400
525518,2019-06-25 18:45:34,6200,2019-06-25 18:53:58,6918,503,1,Maguire / St-Laurent,45.525,-73.596,Garnier / des Carrières,45.538,-73.590,<NA>,24.800
3476615,2019-06-05 18:07:55,6046,2019-06-05 18:16:49,6403,534,1,Métro Bonaventure (de la Gauchetière / de la C...,45.498,-73.567,Georges-Vanier / Notre-Dame,45.485,-73.572,<NA>,16.900
1975639,2019-06-18 20:58:22,6017,2019-06-18 21:05:55,6012,453,1,Atateken / du Square Amherst,45.521,-73.564,Métro St-Laurent (de Maisonneuve / St-Laurent),45.511,-73.565,<NA>,23.500
4340917,2019-04-18 18:55:11,6248,2019-04-18 18:57:36,6211,144,1,St-Dominique / Rachel,45.519,-73.582,Roy / St-Laurent,45.516,-73.576,Rain,11.400
1094433,2019-08-12 14:34:08,6083,2019-08-12 14:47:54,6095,825,1,Square Phillips,45.504,-73.568,Chomedey / de Maisonneuve,45.492,-73.584,<NA>,26.000
5339238,2019-07-31 06:35:14,6903,2019-07-31 06:47:36,7041,742,1,St-Dominique / Napoléon,45.517,-73.577,de Maisonneuve / Peel,45.501,-73.575,<NA>,21.400
1572425,2019-06-27 09:08:18,6023,2019-06-27 09:16:19,6031,481,1,de la Commune / Berri,45.511,-73.550,de Maisonneuve / Robert-Bourassa,45.504,-73.572,<NA>,23.700
563054,2019-07-04 17:34:15,7066,2019-07-04 17:50:24,7050,968,1,St-Urbain / de la Gauchetière,45.507,-73.561,Ottawa / William,45.490,-73.564,<NA>,32.200


## Data Cleaning

### Checking if there are missing data
Only the weather data is missing some values

In [11]:
bixi_2019_all_with_stations_and_weather.isnull().sum()

start_date                  0
start_station_code          0
end_date                    0
end_station_code            0
duration_sec                0
is_member                   0
start_name                  0
start_latitude              0
start_longitude             0
end_name                    0
end_latitude                0
end_longitude               0
start_weather         3459073
start_temp               1494
dtype: int64

### Checking the contents of the "Weather" column
Some of the weather values contains more than 1 condition. We'll have to take this into account while training.

In [12]:
bixi_2019_all_with_stations_and_weather['start_weather'].value_counts()

Mostly Cloudy                              633845
Mainly Clear                               400336
Cloudy                                     304650
Clear                                      281318
Rain                                       165817
Rain Showers                               102038
Rain,Fog                                    20620
Drizzle,Fog                                 17388
Fog                                         16463
Drizzle                                     16140
Thunderstorms,Rain Showers                  11358
Moderate Rain,Fog                            4062
Moderate Rain Showers                        3378
Rain,Rain Showers                            1437
Moderate Rain                                1202
Thunderstorms,Rain                            793
Thunderstorms,Moderate Rain Showers,Fog       635
Thunderstorms,Moderate Rain Showers           612
Thunderstorms,Heavy Rain Showers              577
Thunderstorms                                 274


We will have to find a way to split the weather conditions into smaller parts. 

The best way to do so is to create new features to the data that represent the weather and set the flag to either 0 or 1

Here are the unique weather conditions:
- Mostly Cloudy
- Mainly Clear
- Cloudy
- Clear
- Rain
- Rain Showers
- Fog
- Drizzle
- Moderate Rain Showers
- Moderate Rain
- Heavy Rain Showers
- Thunderstorms

Also, more than half of the data does not have a weather appended to it. I'm not sure if this will affect the results or not. 

## Expanding data

### Weather flags
First, we create new features for the weather data and set the appropriate flag for all trip data

***May take a while to run**

In [13]:
# Using "temp" variable so I can display the results and debug before overwriting the original variable
temp = bixi_2019_all_with_stations_and_weather

temp = temp.join(temp['start_weather'].str.get_dummies(','))

bixi_2019_all_with_stations_and_weather = temp

bixi_2019_all_with_stations_and_weather.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,...,Drizzle,Fog,Heavy Rain Showers,Mainly Clear,Moderate Rain,Moderate Rain Showers,Mostly Cloudy,Rain,Rain Showers,Thunderstorms
5208344,2019-05-26 10:21:26,6166,2019-05-26 10:35:16,6248,829,1,Gerry-Boulet / St-Grégoire,45.534,-73.589,St-Dominique / Rachel,...,0,0,0,0,0,0,1,0,0,0
4533962,2019-10-11 10:56:26,6202,2019-10-11 11:01:54,7046,327,1,Ste-Famille / Sherbrooke,45.510,-73.571,de Maisonneuve / Mansfield (nord),...,0,0,0,0,0,0,0,0,0,0
2703400,2019-09-19 18:07:12,6042,2019-09-19 18:19:23,6013,731,1,de la Gauchetière / Robert-Bourassa,45.501,-73.565,Sanguinet / de Maisonneuve,...,0,0,0,0,0,0,0,0,0,0
3279799,2019-10-16 10:49:23,6729,2019-10-16 11:00:23,6206,660,1,St-André / Ste-Catherine,45.516,-73.558,Prince-Arthur / du Parc,...,0,0,0,0,0,0,0,0,0,0
3158524,2019-06-20 20:09:08,6092,2019-06-20 20:17:22,6073,493,1,Crescent / Ste-Catherine,45.497,-73.576,de Maisonneuve / Aylmer (ouest),...,0,0,0,0,0,0,0,0,0,0


### Splitting start_date to year, month, day, hour, minute, second columns
Also adding seconds since midnight for simpler normalization

In [33]:
temp = bixi_2019_all_with_stations_and_weather

temp['start_year'] = temp['start_date'].dt.year
temp['start_month'] = temp['start_date'].dt.month
temp['start_day'] = temp['start_date'].dt.day
temp['start_hour'] = temp['start_date'].dt.hour
temp['start_minute'] = temp['start_date'].dt.minute
temp['start_second'] = temp['start_date'].dt.second
temp['start_seconds_since_midnight'] = (temp['start_date'] - pd.to_datetime(temp['start_date'].dt.strftime('%Y-%m-%d') + ' ' + '00:00:00')).astype('timedelta64[s]')

bixi_2019_all_with_stations_and_weather = temp

bixi_2019_all_with_stations_and_weather.sample(5)

,start_date,start_station_code,end_date,end_station_code,duration_sec,is_member,start_name,start_latitude,start_longitude,end_name,...,Rain,Rain Showers,Thunderstorms,start_year,start_month,start_day,start_hour,start_minute,start_second,start_seconds_since_midnight
4508849,2019-05-09 07:50:39,6927,2019-05-09 08:12:42,7066,1323,1,Édouard-Montpetit / de Stirling,45.507,-73.615,St-Urbain / de la Gauchetière,...,0,0,0,2019,5,9,7,50,39,28239.000
404749,2019-10-24 08:13:47,6148,2019-10-24 08:27:14,6053,806,1,Émile-Duployé / Sherbrooke,45.527,-73.565,du Président-Kennedy / Jeanne-Mance,...,0,0,0,2019,10,24,8,13,47,29627.000
2706486,2019-05-12 19:58:36,6023,2019-05-12 20:07:44,6099,547,1,de la Commune / Berri,45.511,-73.550,Ste-Catherine / Labelle,...,0,0,0,2019,5,12,19,58,36,71916.000
4205936,2019-09-08 13:02:19,6120,2019-09-08 13:15:42,6157,802,1,Métro Frontenac (Ontario / du Havre),45.534,-73.552,de Brébeuf / du Mont-Royal,...,0,0,0,2019,9,8,13,2,19,46939.000
3568558,2019-06-21 20:12:48,6255,2019-06-21 20:17:51,6262,303,1,Boyer / St-Zotique,45.538,-73.606,de la Roche / de Bellechasse,...,0,0,0,2019,6,21,20,12,48,72768.000


### Adding number of trips in a +/- 30 minutes time frame of that particular station

This will serve as our "y" parameter, aka the value we are trying to predict

In [195]:
temp = bixi_2019_all_with_stations_and_weather
len(temp)

5442288

In [190]:
# I couldn't figure out how to vectorize this...

with tqdm(total=len(temp)) as pbar: # Progress bar
    for i, row in temp.iterrows():
        # Get the current row's station code
        target_station_code = row['start_station_code']
        target_start_date = row['start_date']

        # Retrieve all the data for that particular station code
        df_station = temp[(temp['start_station_code'] == target_station_code)]
        
        # Count how many trips were started +/- 30min of the start time
        sum_count = df_station['start_date'].between(
            target_start_date + pd.Timedelta('-30 minutes'),
            target_start_date + pd.Timedelta('30 minutes')
        ).sum()

        #Assign sum to new trip_count column
        temp.loc[i,'trip_count'] = sum_count
        
        #Update progress bar
        pbar.update(1)

temp.to_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')

  0%|          | 0/1940 [00:00<?, ?it/s]/Users/simon/opt/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
100%|██████████| 1940/1940 [00:03<00:00, 602.86it/s]


In [ ]:
processed = pd.read_pickle('./data/bixi_2019_all_with_stations_and_weather_and_trip_count.pkl')
processed